In [1]:
import pandas as pd
import numpy as np
import googlemaps
import os

### 데이터 불러오기

In [10]:
BNC = pd.read_csv("./data/NSC2_BNC_1619.CSV", usecols=[ "RN_INDI", "STD_YYYY", "SGG"])            # RN_INDI, STD_YYYY, SGG
INST = pd.read_csv("./data/NSC2_INST_1619.CSV", usecols=["STD_YYYY", "RN_INST", "SGG_INST"])     # STD_YYYY, RN_INST, SGG_INST

In [11]:
T20_dir = "./data/20T"
T20_list= os.listdir(T20_dir)
T20 = pd.DataFrame(columns=["RN_INDI", "RN_INST", "MCARE_SUBJ_CD", "STD_YYYY"])
for name in T20_list:
    print(name + "불러오기 시작")
    temp = pd.read_csv(T20_dir + "/" + name, usecols=["RN_INDI", "RN_INST", "MCARE_SUBJ_CD", "STD_YYYY"])  # RN_INDI, RN_INST,MCARE_SUBJ_CD, STD_YYYY
    temp.drop(index=0, axis=0)
    T20 = pd.concat([T20, temp])
    print("완료")
T20 = T20[["STD_YYYY", "RN_INDI", "RN_INST", "MCARE_SUBJ_CD"]]

NSC2_D20_1619.CSV불러오기 시작
완료
NSC2_K20_1619.CSV불러오기 시작
완료
NSC2_M20_1619.CSV불러오기 시작
완료
NSC2_P20_1619.CSV불러오기 시작
완료


### 중복제거 및 2019년 필터링

In [15]:
# BNC = BNC.drop_duplicates("RN_INDI")
INST = INST.drop_duplicates("RN_INST").sort_values(by="STD_YYYY", axis=0, ascending=False).reset_index(drop=True)
BNC = BNC[["STD_YYYY", "RN_INDI", "SGG"]]

BNC2019 = BNC[BNC["STD_YYYY"] == 2019].sort_values(by=['RN_INDI'], axis=0).reset_index(drop=True)
INST2019 = INST[INST["STD_YYYY"] == 2019].sort_values(by=['RN_INST'], axis=0).reset_index(drop=True)
T202019 = T20[T20["STD_YYYY"] == 2019].sort_values(by=['RN_INDI'], axis=0).reset_index(drop=True)

In [16]:
print("BNC")
print(BNC2019.head())

print("\nINST")
print(INST2019.head())

print("\nT20")
print(T202019)

BNC
   STD_YYYY  RN_INDI    SGG
0      2019     1433  41190
1      2019     3085  29200
2      2019     4316  28177
3      2019     4392  29200
4      2019     6088  27290

INST
   STD_YYYY  RN_INST  SGG_INST
0      2019        1     11290
1      2019        2     11230
2      2019        4     41111
3      2019        6     11260
4      2019       11     11140

T20
      STD_YYYY  RN_INDI RN_INST MCARE_SUBJ_CD
0         2019     1433   79512             0
1         2019     1433   79512             0
2         2019     1433   79512             0
3         2019     1433   79512             0
4         2019     1433   79512             0
...        ...      ...     ...           ...
27322     2019  3000758  161290             0
27323     2019  3000758   52897            11
27324     2019  3000758   52897            11
27325     2019  3000758  161290             0
27326     2019  3000758   93343            12

[27327 rows x 4 columns]


### 결과데이터에 개인주소지 및 기관주소지 추가 + 이상치( 정보데이터가 없거나 주소지와 방문기관주소지가 같은 관측치 ) 제거

In [17]:
inputData = T202019.copy()
compare_BNC = BNC2019.copy()
compare_INST = INST2019.copy()
compare_INST = INST.copy()

result_SSG = []
result_SSG_INST = []
print(str(len(inputData)) + " 개의 데이터가 있습니다.")
print("처리중", end='')

for idx in range(len(inputData)):
    if(idx%1000==0):
        print(".", end='')
    localValue_INDI = inputData.iloc[idx]["RN_INDI"]
    localValue_INST = inputData.iloc[idx]["RN_INST"]
    SGG_idx = compare_BNC.index[(compare_BNC["RN_INDI"] == localValue_INDI)][0]

    # 값 없는 부분 예외처리
    try:
        SGG_INST_idx = compare_INST.index[(compare_INST["RN_INST"] == localValue_INST)][0]
    except:
        SGG_INST_idx = -1

    # 결과 데이터 저장( Append )
    result_SSG.append(compare_BNC.iloc[SGG_idx]["SGG"])

    if(SGG_INST_idx != -1):
        result_SSG_INST.append(compare_INST.iloc[SGG_INST_idx]["SGG_INST"])
    else:
        result_SSG_INST.append("-1")

result_data = inputData.copy()
result_data["SSG"] = result_SSG
result_data["SSG_INST"] = result_SSG_INST

delete_idx = result_data.index[(result_data["SSG_INST"] == "-1")]
result_data = result_data.drop(delete_idx)

delete_idx = result_data.index[(result_data["SSG"] == result_data["SSG_INST"])]
result_data = result_data.drop(delete_idx).reset_index(drop=True)

print("완료")

27327 개의 데이터가 있습니다.
처리중............................완료


### 파일저장

In [98]:
# result_data.to_json("test.json", orient='table')

### 위도 경도 데이터 구성


In [18]:
import math

In [75]:
data_base = result_data.copy()
location_table = pd.read_excel("./data/SSG_processed.xlsx").astype({"SSG": int})

print(data_base.head())
print(location_table.head())

  STD_YYYY RN_INDI RN_INST MCARE_SUBJ_CD    SSG  SSG_INST
0     2019    1433   79512             0  41190     41195
1     2019    1433   79512             0  41190     41195
2     2019    1433   79512             0  41190     41195
3     2019    1433   79512             0  41190     41195
4     2019    1433   79512             0  41190     41195
     SSG     시도  시군구        lat         lng
0  11110  서울특별시  종로구  37.572950  126.979358
1  11140  서울특별시   중구  37.564091  126.997940
2  11170  서울특별시  용산구  37.538427  126.965444
3  11200  서울특별시  성동구  37.563341  127.037103
4  11215  서울특별시  광진구  37.538484  127.082294


In [74]:
INST_SSG = data_base.iloc[idx]["SSG_INST"]
print(INST_SSG)

28177


In [72]:
location_table.index[(location_table["SSG"] == INDI_SSG)][0]

241

In [76]:
distance = []

print(str(len(data_base)) + " 개의 데이터가 있습니다.")
print("처리중", end='')
for idx in range(len(data_base)):
    if(idx%500 == 0):
        print(".", end='')
    INDI_SSG = data_base.iloc[idx]["SSG"]
    INDI_SGG_idx = location_table.index[(location_table["SSG"] == INDI_SSG)][0]
    INDI_lat = location_table.iloc[INDI_SGG_idx]["lat"]
    INDI_lng = location_table.iloc[INDI_SGG_idx]["lng"]

    INST_SSG = data_base.iloc[idx]["SSG_INST"]
    INST_SGG_idx = location_table.index[(location_table["SSG"] == INST_SSG)][0]
    INST_lat = location_table.iloc[INST_SGG_idx]["lat"]
    INST_lng = location_table.iloc[INST_SGG_idx]["lng"]

    cal_result = math.sqrt(((INDI_lat-INST_lat)*111)**2 + ((INDI_lng-INST_lng)*88.8)**2)
    # print( "{( " + str(INDI_lat) + " - " + str(INST_lat) + " ) x 111}^2 {(" + str(INDI_lng) + " - " + str(INST_lng) + ") x 88.8 }**2" )

    distance.append(round(cal_result,3))
print("끝!")
result = data_base.copy()
result["DISTANCE"] = distance

9730 개의 데이터가 있습니다.
처리중....................끝!


In [78]:
print(result.head())
result.to_excel("Analysis Result.xlsx")

  STD_YYYY RN_INDI RN_INST MCARE_SUBJ_CD    SSG  SSG_INST  DISTANCE
0     2019    1433   79512             0  41190     41195       0.0
1     2019    1433   79512             0  41190     41195       0.0
2     2019    1433   79512             0  41190     41195       0.0
3     2019    1433   79512             0  41190     41195       0.0
4     2019    1433   79512             0  41190     41195       0.0


In [81]:
analysis = pd.read_excel('./data/Result_base.xlsx')
analysis.head()

,SSG,시도,시군구
0,11110,서울특별시,종로구
1,11140,서울특별시,중구
2,11170,서울특별시,용산구
3,11200,서울특별시,성동구
4,11215,서울특별시,광진구


In [82]:
result.head()

,STD_YYYY,RN_INDI,RN_INST,MCARE_SUBJ_CD,SSG,SSG_INST,DISTANCE
0,2019,1433,79512,0,41190,41195,0.0
1,2019,1433,79512,0,41190,41195,0.0
2,2019,1433,79512,0,41190,41195,0.0
3,2019,1433,79512,0,41190,41195,0.0
4,2019,1433,79512,0,41190,41195,0.0


In [ ]:
for idx in len(result):
    src_SSG = result.iloc[idx]["SSG"]
    analysys_idx = analysis.index[(analysis["SSG"] == src_SSG)][0]

    dst_SSG = result.iloc[idx]["SSG"]
    analysys_idx = analysis.index[(analysis["SSG"] == src_SSG)][0]


